In [1]:
import requests as rq
import csv
import json
from flask import request, jsonify
import os
from datetime import datetime
import subprocess

## try this again, with the nyt data

- find the most recent record by county, then take out fips, cases, deaths

In [17]:
def geojson_to_dict(path):
    with open(path, "r") as f:
        counties_coords = json.load(f)
    return counties_coords

In [13]:
def csv_to_dict(path):
    with open(path, "r") as f:
        csv_file = csv.reader(f, delimiter=",")
        csv_unpacked = [row for row in csv_file]
        states_coords = [{csv_unpacked[0][i]: row[i] for i in range(len(row))} for row in csv_unpacked][1:]
        
    return states_coords

In [14]:
def pull_nyt_data():
    return subprocess.check_output(["git", "-C", "covid-19-data", "pull"])

In [15]:
def recent_covid_results(): 
    PATH_COUNTIES = os.path.join(os.getcwd(), 'covid-19-data', "us-counties.csv")

    counties_coords = geojson_to_dict('us_counties_coords.geojson')
    counties_covid = csv_to_dict(PATH_COUNTIES)
    
    counties_covid_recent = {}
    for county in counties_covid:
        curr_keys = counties_covid_recent.keys()
        this_fip = county['fips']
        this_deaths = county['deaths']
        this_cases = county['cases']
        this_date = datetime.strptime(county['date'], "%Y-%m-%d")

        if this_fip in curr_keys and this_date > counties_covid_recent[this_fip]['date']:
            counties_covid_recent[this_fip] = {'date':this_date, 'cases':this_cases, 'deaths':this_deaths}
        elif this_fip not in curr_keys:
            counties_covid_recent[this_fip] = {'date':this_date, 'cases':this_cases, 'deaths':this_deaths}

    for county_poly in counties_coords["features"]:
        for covid_key, covid_value in counties_covid_recent.items():
            if county_poly['properties']['FIPS']==covid_key:
                county_poly['properties']['cases']=covid_value['cases']
                county_poly['properties']['deaths']=covid_value['deaths']
                break
                
    return counties_coords

In [73]:
demo_state = []
for county in counties_coords['features']:
    if county['properties']['STATE_NAME']=='California':
        demo_state.append(county)

In [74]:
demo_state

[{'type': 'Feature',
  'properties': {'NAME': 'Siskiyou',
   'STATE_NAME': 'California',
   'STATE_FIPS': '06',
   'CNTY_FIPS': '093',
   'FIPS': '06093',
   'cases': '5',
   'deaths': '0'},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-121.44473687605502, 41.18080100968611],
      [-122.48918028721766, 41.179704059151085],
      [-122.51528905685618, 41.20085243660164],
      [-122.51613594929285, 41.21172434314155],
      [-122.49947349083305, 41.29607862188966],
      [-122.4776529041668, 41.3147403018393],
      [-122.50689773494054, 41.33621628803524],
      [-122.52980967730554, 41.33932105953065],
      [-122.54394777672775, 41.363254007708875],
      [-122.57229960946971, 41.365206657367274],
      [-122.58505583614, 41.32509924478073],
      [-122.63727311639681, 41.319170479683635],
      [-122.64887742844624, 41.28820306099064],
      [-122.70880055956889, 41.27190224653014],
      [-122.7262873757082, 41.24611497276037],
      [-122.76581655160658, 41.227971